In [80]:
import pandas as pd
import numpy as np
import os
import sys

### Read the data

In [81]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [82]:
class HailStone:
    def __init__(self, px, py, vx, vy):
        self.px = int(px)
        self.py = int(py)
        self.vx = int(vx)
        self.vy = int(vy)

        self.a = self.vy / self.vx
        self.b = self.py - self.a * self.px

    def update_position_one_ns(self):
        self.px += self.vx
        self.py += self.vy

    def has_intersection(self, other, min_area=-np.inf, max_area=np.inf):
        if self.a == other.a:
            return False
        intersect_x = (other.b - self.b) / (self.a - other.a)
        intersect_y = self.a * intersect_x + self.b
        if intersect_x < min_area or intersect_x > max_area:
            return False
        if intersect_y < min_area or intersect_y > max_area:
            return False
        # check if the intersection is in the past of self
        if intersect_x < self.px and self.vx > 0:
            return False
        if intersect_x > self.px and self.vx < 0:
            return False
        # check if the intersection is in the past of other
        if intersect_x < other.px and other.vx > 0:
            return False
        if intersect_x > other.px and other.vx < 0:
            return False
        return True

    def __repr__(self):
        return f'HailStone({self.px}, {self.py} @ {self.vx}, {self.vy})'

    def __eq__(self, other):
        return self.px == other.px and self.py == other.py


class HailStorm:
    def __init__(self):
        self.hailstones = list()

    def add_hailstone(self, hailstone):
        self.hailstones.append(hailstone)

    def get_hailstones(self):
        return self.hailstones

    def update_positions_one_ns(self):
        for hailstone in self.hailstones:
            hailstone.update_position_one_ns()

    def __repr__(self):
        return_str = ''
        for hailstone in self.hailstones:
            return_str += f'{hailstone}\n'
        return return_str
            

def process_file(working_file):
    hailstorm = HailStorm()
    for line in working_file:
        # remove spaces from line
  
        px, py, _, vx, vy, _ = line.replace(' ', '').replace('@', ',').split(',')

        hailstone = HailStone(px, py, vx, vy)
        hailstorm.add_hailstone(hailstone)
    return hailstorm

In [83]:
working_file = file_content

hailstorm = process_file(working_file)

hailstones = hailstorm.get_hailstones()

n_intersections = 0
for i, hailstone in enumerate(hailstones):
    # iterate only on the hailstones after the current one
    for j, other_hailstone in enumerate(hailstones[i+1:]):
        n_intersections += hailstone.has_intersection(other_hailstone, 200000000000000, 400000000000000)


print(n_intersections)



31921


### Part 2

In [85]:
import sys
sys.setrecursionlimit(1000000)
ans = res = 0

with open("day24.txt") as f:
    s = f.read().strip()

d = []
for line in s.split("\n"):
    pos, vel = line.split(" @ ")
    pos = list(map(int, pos.split(", ")))
    vel = list(map(int, vel.split(", ")))
    d.append((pos, vel))



for i in range(len(d)):
    for j in range(i+1,len(d)):
        as_,ad = d[i]
        bs,bd = d[j]


        try:
            u = ((bs[1] - as_[1]) * bd[0] - (bs[0] - as_[0]) * bd[1]) / (bd[0] * ad[1] - bd[1] * ad[0])
            v = ((bs[1] - as_[1]) * ad[0] - (bs[0] - as_[0]) * ad[1]) / (bd[0] * ad[1] - bd[1] * ad[0])

            if u < 0 or v < 0:
                continue
            xi = bs[0] + bd[0] * v
            yi = bs[1] + bd[1] * v
            if xi >= 200000000000000 and xi <= 400000000000000 and \
               yi >= 200000000000000 and yi <= 400000000000000:
                ans += 1
        except:
            pass
   
print(ans)

# ox, oy, oz, mvx, mvy, mvz
# o, m

# px, py, pz, vx, vy, vz
# p, v

# o + ti * m - pi - ti * vi = 0
# o + ti * m - pi - ti * vi = 0

# ox - pxi + ti * (mvx - vxi) = 0
# oy - pyi + ti * (mvy - vyi) = 0
# oz - pzi + ti * (mvz - vzi) = 0

# ox, oy, oz, ti
import numpy as np

def get_mats(mvx, mvy, mvz):
    A = []
    b = []
    for i,((px,py,pz),(vx,vy,vz)) in enumerate(d):
        eqn = [1, 0, 0] + [0] * len(d)
        eqn[3+i] = mvx - vx
        A.append(tuple(eqn))
        b.append(px)
        
        eqn = [0, 1, 0] + [0] * len(d)
        eqn[3+i] = mvy - vy
        A.append(tuple(eqn))
        b.append(py)
        
        eqn = [0, 0, 1] + [0] * len(d)
        eqn[3+i] = mvz - vz
        A.append(tuple(eqn))
        b.append(pz)

    A = np.matrix(A)
    b = np.matrix(b).T
    return A, b

def solve_given_v(mvx, mvy, mvz, retthing=1):
    A, b = get_mats(mvx, mvy, mvz)
    return np.linalg.lstsq(A, b, rcond=None)[retthing]


vx, vy, vz = 0, 0, 0
cur_score = float(solve_given_v(vx, vy, vz))
step_size = 0.0001
lr = 1e-22
not_improve = 0
close_ctr = 0
while close_ctr < 10:
    dvx = float(solve_given_v(vx + step_size, vy, vz)) - cur_score
    dvy = float(solve_given_v(vx, vy + step_size, vz)) - cur_score
    dvz = float(solve_given_v(vx, vy, vz + step_size)) - cur_score

    vx -= lr * dvx
    vy -= lr * dvy
    vz -= lr * dvz

    last_score = cur_score
    cur_score = float(solve_given_v(vx, vy, vz))
    if cur_score > last_score:
        not_improve += 1

    if not_improve > 5:
        not_improve = 0
        lr = lr / 2

    if all(((vi + 0.5) % 1) - 0.5 < 0.001 for vi in (vx,vy,vz)):
        close_ctr += 1
    else:
        close_ctr = 0

vx = round(vx)
vy = round(vy)
vz = round(vz)

A, b = get_mats(vx,vy,vz)

res = np.linalg.solve(A[:5,:5], b[:5])
print(res[0,0] + res[1,0] + res[2,0])

31921


C:\Users\j.schilperoort\AppData\Local\Temp\ipykernel_16904\1564427606.py:84: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cur_score = float(solve_given_v(vx, vy, vz))
C:\Users\j.schilperoort\AppData\Local\Temp\ipykernel_16904\1564427606.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dvx = float(solve_given_v(vx + step_size, vy, vz)) - cur_score
C:\Users\j.schilperoort\AppData\Local\Temp\ipykernel_16904\1564427606.py:91: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1

761691907059631.0
